### Inference on the trained model ###

In [7]:
import sys
import os
import numpy as np
import json
import pandas as pd
from pathlib import Path
from matplotlib import pyplot as plt
from matplotlib import patches

# PyTorch
import torch

# Hugging Face Library
from transformers import RTDetrV2ForObjectDetection, RTDetrImageProcessor

%load_ext autoreload
%autoreload 2
import computervision
from computervision.imageproc import is_image
from computervision.datasets import DTRdataset, get_gpu_info
from computervision.transformations import AugmentationTransform

print(f'Project version: {computervision.__version__}')
print(f'Python version:  {sys.version}')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Project version: v0.0.1
Python version:  3.12.3 (main, Jun 18 2025, 17:59:45) [GCC 13.3.0]


In [3]:
# Check GPU availability
device, device_str = get_gpu_info()

CUDA available: True
Number of GPUs found:  1
Current device ID: 0
GPU device name:   NVIDIA GeForce RTX 3070 Laptop GPU
PyTorch version:   2.8.0a0+34c6371d24.nv25.08
CUDA version:      13.0
CUDNN version:     91200


In [14]:
# Directories and files
data_dir = os.path.join(os.environ.get('HOME'), 'data')
train_image_dir = os.path.join(data_dir, 'dentex', 'cropped')
val_image_dir = os.path.join(train_image_dir, 'test')
model_dir = os.path.join(data_dir, 'model')

train_annotation_file_name = 'train_quadrant_enumeration_dset.parquet'
train_annotation_file = os.path.join(train_image_dir, train_annotation_file_name)
val_annotation_file_name = 'train_quadrant_enumeration_test_set.parquet'
val_annotation_file = os.path.join(val_image_dir, val_annotation_file_name)



# Column names for the annotation files
label_col = 'label'
label_name_col = 'ada'
file_name_col = 'file_name'
bbox_col = 'bbox'
dset_col = 'dset'

# Model name
model_name = 'rtdetr_250925_01'

# Model parameter file
json_file = os.path.join(model_dir, f'{model_name}.json')
with open(json_file, mode='r') as fl:
    parameters = json.load(fl)

print(parameters.keys())
model_info = parameters.get('model_info')
processor_params = parameters.get('processor_params')

dict_keys(['model_info', 'id2label', 'training_args', 'processor_params'])


In [19]:
id2label = parameters.get('id2label')
label2id = {v: k for k, v in id2label.items()}
hf_checkpoint = model_info.get('hf_checkpoint')

# Load the image processor
image_processor = RTDetrImageProcessor.\
    from_pretrained(hf_checkpoint, **processor_params)

# Load the model from checkpoint
checkpoint_name = 'checkpoint-10140'
checkpoint_file = os.path.join(model_dir, checkpoint_name)
model = RTDetrV2ForObjectDetection.from_pretrained(checkpoint_file).to(device)

HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/app/data/model/checkpoint-10140'. Use `repo_type` argument if needed.

In [16]:
model_info

{'model_version': 1,
 'project_version': 'v0.0.1',
 'model_name': 'rtdetr_250925_01',
 'model_dir': '/app/data/model',
 'train_image_dir': '/app/data/dentex/cropped',
 'val_image_dir': '/app/data/dentex/cropped/test',
 'image_width': 640,
 'image_height': 640,
 'hf_checkpoint': 'PekingU/rtdetr_v2_r101vd',
 'train_transform': 'train_1',
 'val_transform': 'val_1'}

In [15]:
processor_params

{'do_resize': True,
 'size': {'max_height': 640, 'max_width': 640},
 'do_pad': True,
 'pad_size': {'height': 640, 'width': 640}}